<a href="https://colab.research.google.com/github/baijuthomas528/python-training/blob/main/Curve_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import plotly.express as px

# Step 1: Upload the file from your computer
uploaded = files.upload()   # this will open a "Choose File" dialog


Saving Curve_data.xlsm to Curve_data (5).xlsm


In [ ]:
# ==== 0) Upload (re-run this cell to upload a new file) ====
from google.colab import files, output
output.enable_custom_widget_manager()
uploaded = files.upload()   # choose Curve_data.xlsm

# ==== 1) Imports ====
import pandas as pd
import numpy as np
import plotly.express as px
from ipywidgets import Dropdown, SelectMultiple, Button, HBox, VBox, Output, HTML
from IPython.display import display, clear_output

# ==== 2) Load workbook ====
df = pd.read_excel("Curve_data.xlsm", sheet_name=0)

# ==== 3) Clean / prepare ====
all_markets = ['CA','AH','AA','NA','NI','PB','SN','ZS','CO']
markets = [c for c in all_markets if c in df.columns]

# Coerce numerics for price columns
if markets:
    df[markets] = df[markets].apply(pd.to_numeric, errors='coerce')

# Normalize Date column name and parse dates (dayfirst=True for UK style)
date_col = next((c for c in df.columns if str(c).strip().lower() == "date"), None)
if date_col is None:
    raise ValueError("Could not find a 'Date' column.")
df = df.rename(columns={date_col: 'Date'})
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)

# Create Day count per Date if missing (1..N), sorting by Prompt if available
if 'Day count' not in df.columns:
    sort_cols = ['Date']
    if 'Prompt' in df.columns:
        df['__PromptNum'] = pd.to_numeric(df['Prompt'], errors='coerce')
        sort_cols.append('__PromptNum')
    df = df.sort_values(sort_cols, kind='mergesort')
    df['Day count'] = df.groupby('Date').cumcount() + 1
    df.drop(columns=['__PromptNum'], errors='ignore', inplace=True)
else:
    df['Day count'] = pd.to_numeric(df['Day count'], errors='coerce')

# Drop unusable rows
df = df.dropna(subset=['Date','Day count'])

# Pivot to (Market, Day count)
wide = df.set_index(['Date','Day count'])[markets].unstack('Day count')

# Ensure Day count level is numeric on the columns MultiIndex
lvl_mkt, lvl_day = wide.columns.levels
lvl_day_num = pd.to_numeric(lvl_day, errors='coerce')
wide.columns = pd.MultiIndex.from_product([lvl_mkt, lvl_day_num], names=wide.columns.names)

# Day options & defaults
available_days = sorted(set(
    wide.columns.get_level_values(1)[~pd.isna(wide.columns.get_level_values(1))].astype(int)
))
default_base = 1 if 1 in available_days else available_days[0]
default_comp = 90 if 90 in available_days else available_days[-1]

# Date dropdowns: use ONLY dates present in the data
date_index = wide.index.get_level_values(0).unique().sort_values()
date_options = [(ts.strftime('%d-%b-%Y'), ts) for ts in date_index]

# ==== 4) Widgets (Run button) ====
base_dd   = Dropdown(options=available_days, value=default_base, description='Base Day')
comp_dd   = Dropdown(options=available_days, value=default_comp, description='Compare Day')
start_dd  = Dropdown(options=date_options, value=date_index.min(), description='Start Date')
end_dd    = Dropdown(options=date_options, value=date_index.max(), description='End Date')
assets_ms = SelectMultiple(options=markets, value=tuple(markets if markets else []),
                           description='Assets', rows=min(10, len(markets)))
run_btn   = Button(description='Run', button_style='primary', icon='play', tooltip='Render chart')

out = Output()

def make_title(base_day, compare_day, selected_assets):
    if not selected_assets:
        asset_text = "(no assets selected)"
    elif len(selected_assets) <= 5:
        asset_text = ", ".join(selected_assets)
    else:
        asset_text = f"{len(selected_assets)} assets selected"
    # Explicitly state Base − Compare in the title
    return f"Day {base_day} − Day {compare_day} (Base − Compare) — {asset_text}"

def on_run_clicked(_):
    with out:
        clear_output(wait=True)

        if not assets_ms.value:
            display(HTML("<b>Please select at least one asset.</b>"))
            return

        base_day = int(base_dd.value)
        comp_day = int(comp_dd.value)

        # Check day counts exist
        days_have = set(wide.columns.get_level_values(1).dropna().astype(int))
        missing = {base_day, comp_day} - days_have
        if missing:
            display(HTML(f"<b>Missing day count(s): {sorted(missing)}.<br>"
                         f"Available: {sorted(available_days)}</b>"))
            return

        # Compute differences once, then filter by dates
        sel_assets = list(assets_ms.value)
        d_base = wide.xs(base_day, level=1, axis=1)[sel_assets]
        d_comp = wide.xs(comp_day, level=1, axis=1)[sel_assets]
        diff = (d_base - d_comp)  # <<< Base − Compare

        # Date range from dropdowns (exact dates from the data)
        start = pd.Timestamp(start_dd.value)
        end   = pd.Timestamp(end_dd.value)
        if start > end:
            start, end = end, start

        diff = diff.loc[(diff.index >= start) & (diff.index <= end)].copy()
        # Label columns with Base−Compare notation
        diff.columns = [f"{c}_D{base_day}_minus_D{comp_day}" for c in diff.columns]

        # Long for Plotly
        long_diff = diff.reset_index().melt(id_vars='Date', var_name='Market', value_name='Diff')

        # Single time-series chart
        fig = px.line(
            long_diff,
            x='Date',
            y='Diff',
            color='Market',
            title=make_title(base_day, comp_day, sel_assets),
            markers=True
        )
        fig.update_layout(hovermode='x unified', template='plotly_white')
        display(fig)

run_btn.on_click(on_run_clicked)

# Show UI
display(VBox([HBox([base_dd, comp_dd, start_dd, end_dd, run_btn]), assets_ms]))
display(out)

# Use:
# - Pick Base/Compare Day
# - Pick Start/End Date (from the source Date column)
# - Select assets (Ctrl/Cmd for multi-select)
# - Click "Run" → Base − Compare plot


In [ ]:
# ✅ Step 1: Install required packages
!pip install -q plotly ipywidgets openpyxl


In [ ]:

# ✅ Step 2: Import libraries
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files

# ✅ Step 3: Upload Excel file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# ✅ Step 4: Load data and prepare
xls = pd.ExcelFile(file_name)
df = xls.parse('Sheet1')

# Ensure 'Date' is datetime and compute Day Count
df['Date'] = pd.to_datetime(df['Date'])
df['Day Count'] = df.groupby('Date').cumcount() + 1

# ✅ Step 5: Set up widgets
asset_selector = widgets.Dropdown(
    options=[col for col in df.columns if col not in ['Date', 'Prompt', 'Day Count']],
    description='Asset:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

base_day_selector = widgets.BoundedIntText(
    value=1, min=1, max=int(df['Day Count'].max()), step=1,
    description='Base Day:', layout=widgets.Layout(width='300px')
)

compare_day_selector = widgets.BoundedIntText(
    value=2, min=1, max=int(df['Day Count'].max()), step=1,
    description='Compare Day:', layout=widgets.Layout(width='300px')
)

start_date_selector = widgets.DatePicker(
    description='Start Date',
    value=df['Date'].min().date(),
    layout=widgets.Layout(width='300px')
)

end_date_selector = widgets.DatePicker(
    description='End Date',
    value=df['Date'].max().date(),
    layout=widgets.Layout(width='300px')
)

run_button = widgets.Button(description="Run", button_style='primary')

# ✅ Step 6: Layout
ui = widgets.VBox([
    asset_selector,
    base_day_selector,
    compare_day_selector,
    start_date_selector,
    end_date_selector,
    run_button
])

output = widgets.Output()

display(ui, output)

# ✅ Step 7: Callback function for Run button
def on_run_button_clicked(b):
    with output:
        clear_output(wait=True)

        asset = asset_selector.value
        base_day = base_day_selector.value
        compare_day = compare_day_selector.value
        start_date = pd.to_datetime(start_date_selector.value)
        end_date = pd.to_datetime(end_date_selector.value)

        # Filter data by date
        df_filtered = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

        base_df = df_filtered[df_filtered['Day Count'] == base_day]
        compare_df = df_filtered[df_filtered['Day Count'] == compare_day]

        # Merge on Date and Prompt
        merged = pd.merge(
            base_df[['Date', 'Prompt', asset]].rename(columns={asset: 'Base Value'}),
            compare_df[['Date', 'Prompt', asset]].rename(columns={asset: 'Compare Value'}),
            on=['Date', 'Prompt']
        )

        if merged.empty:
            print("⚠️ No matching data for selected Base Day and Compare Day within date range.")
            return

        merged['Diff'] = merged['Compare Value'] - merged['Base Value']

        # Plot
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=merged['Prompt'],
            y=merged['Diff'],
            mode='lines+markers',
            text=[
                f"Date: {row['Date'].date()}<br>"
                f"Prompt: {row['Prompt']}<br>"
                f"Base Day: {base_day}<br>"
                f"Compare Day: {compare_day}"
                for _, row in merged.iterrows()
            ],
            hoverinfo='text',
            name=f'Day {compare_day} - Day {base_day}'
        ))

        fig.update_layout(
            title=f'{asset}: Compare Day {compare_day} - Base Day {base_day}',
            xaxis_title='Prompt Date',
            yaxis_title='Difference in Prices',
            hovermode='closest'
        )

        fig.show()

# ✅ Step 8: Bind button
run_button.on_click(on_run_button_clicked)


Saving Curve_data.xlsm to Curve_data (1).xlsm


Output()

In [ ]:
import numpy as np
import pandas as pd

# ----- Helpers -----
def _parse_csv_list(s):
    return [x.strip() for x in s.split(",") if x.strip()]

def _to_float_list(xs):
    return [float(x) for x in xs]

def _to_int_list(xs):
    return [int(float(x)) for x in xs]  # tolerate "100.0"

# ----- Core calc -----
def calculate_share_pnl_portfolio(
    tickers, S_list, lots_list,
    percentages=None,
    shock_pct=None,          # e.g. 0.02  (2%)
    two_sided=True,          # if True, adds both +shock and -shock
    show_breakdown=False
):
    """
    tickers: list[str]
    S_list: list[float]   (spot per share, same order as tickers)
    lots_list: list[int]  (shares per ticker, same order as tickers)
    percentages: optional list of scenario % moves (e.g., [-0.2, -0.15, ..., 0.2])
    shock_pct: optional float or list to add as extra shock(s), e.g. 0.02 for 2%
    two_sided: if True and shock_pct is a single number, add both ±shock
    """
    # default grid
    if percentages is None:
        percentages = [-0.20, -0.15, -0.10, -0.08, -0.05, 0, 0.05, 0.08, 0.10, 0.15, 0.20]

    # add shock(s)
    if shock_pct is not None:
        if isinstance(shock_pct, (int, float)):
            add = [float(shock_pct)]
            if two_sided and float(shock_pct) != 0:
                add.append(-float(shock_pct))
        else:
            add = [float(x) for x in shock_pct]  # allow a list
        # de-dup and sort nicely
        percentages = sorted(set(round(x, 6) for x in (percentages + add)))

    if not (len(tickers) == len(S_list) == len(lots_list)):
        raise ValueError("tickers, S_list, and lots_list must have the same length.")

    initial_values = [S * q for S, q in zip(S_list, lots_list)]
    initial_portfolio_value = sum(initial_values)

    # Consolidated portfolio PnL across scenarios
    portfolio_rows = []
    for pct in percentages:
        new_values = [(S * (1 + pct)) * q for S, q in zip(S_list, lots_list)]
        new_portfolio_value = sum(new_values)
        pnl = new_portfolio_value - initial_portfolio_value
        portfolio_rows.append({
            "Percentage Change": pct,
            "Portfolio Value (New)": new_portfolio_value,
            "Initial Portfolio Value": initial_portfolio_value,
            "PnL (Portfolio)": pnl
        })

    df_portfolio = pd.DataFrame(portfolio_rows)
    # Pretty formatting
    df_portfolio["Percentage Change"] = df_portfolio["Percentage Change"].apply(lambda x: f"{x*100:.2f}%")
    df_portfolio["Portfolio Value (New)"] = df_portfolio["Portfolio Value (New)"].apply(lambda x: f"${x:,.2f}")
    df_portfolio["Initial Portfolio Value"] = df_portfolio["Initial Portfolio Value"].apply(lambda x: f"${x:,.2f}")
    df_portfolio["PnL (Portfolio)"] = df_portfolio["PnL (Portfolio)"].apply(lambda x: f"${x:,.2f}")

    if show_breakdown:
        breakdown_rows = []
        for tkr, S, q in zip(tickers, S_list, lots_list):
            init_val = S * q
            for pct in percentages:
                new_val = (S * (1 + pct)) * q
                pnl = new_val - init_val
                breakdown_rows.append({
                    "Ticker": tkr,
                    "Percentage Change": pct,
                    "New Spot Price": S * (1 + pct),
                    "Shares": q,
                    "Initial Position Value": init_val,
                    "PnL": pnl
                })
        df_breakdown = pd.DataFrame(breakdown_rows)
        df_breakdown["Percentage Change"] = df_breakdown["Percentage Change"].apply(lambda x: f"{x*100:.2f}%")
        df_breakdown["New Spot Price"] = df_breakdown["New Spot Price"].apply(lambda x: f"${x:,.2f}")
        df_breakdown["Initial Position Value"] = df_breakdown["Initial Position Value"].apply(lambda x: f"${x:,.2f}")
        df_breakdown["PnL"] = df_breakdown["PnL"].apply(lambda x: f"${x:,.2f}")
        return df_portfolio, df_breakdown

    return df_portfolio

# ===== Example inputs =====
tickers_csv = "AHD, Copper"
S_csv       = "2620, 9780"
lots_csv    = "-79650, -16250"

tickers   = _parse_csv_list(tickers_csv)
S_list    = _to_float_list(_parse_csv_list(S_csv))
lots_list = _to_int_list(_parse_csv_list(lots_csv))

# ----- Run with a ±2% shock added -----
df_port = calculate_share_pnl_portfolio(
    tickers, S_list, lots_list,
    shock_pct=0.02,     # adds both +2% and -2% by default
    two_sided=True,
    show_breakdown=False
)

print(f"Tickers: {', '.join(tickers)}")
print("Consolidated Portfolio PnL incl. ±2% shock:")
print(df_port.to_string(index=False))

# If you want ONLY a +2% shock (no -2%):
# df_port = calculate_share_pnl_portfolio(tickers, S_list, lots_list, shock_pct=0.02, two_sided=False)

# If you want ONLY a -2% shock:
# df_port = calculate_share_pnl_portfolio(tickers, S_list, lots_list, shock_pct=-0.02, two_sided=False)


Tickers: AHD, Copper
Consolidated Portfolio PnL incl. ±2% shock:
Percentage Change Portfolio Value (New) Initial Portfolio Value PnL (Portfolio)
          -20.00%      $-294,086,400.00        $-367,608,000.00  $73,521,600.00
          -15.00%      $-312,466,800.00        $-367,608,000.00  $55,141,200.00
          -10.00%      $-330,847,200.00        $-367,608,000.00  $36,760,800.00
           -8.00%      $-338,199,360.00        $-367,608,000.00  $29,408,640.00
           -5.00%      $-349,227,600.00        $-367,608,000.00  $18,380,400.00
           -2.00%      $-360,255,840.00        $-367,608,000.00   $7,352,160.00
            0.00%      $-367,608,000.00        $-367,608,000.00           $0.00
            2.00%      $-374,960,160.00        $-367,608,000.00  $-7,352,160.00
            5.00%      $-385,988,400.00        $-367,608,000.00 $-18,380,400.00
            8.00%      $-397,016,640.00        $-367,608,000.00 $-29,408,640.00
           10.00%      $-404,368,800.00        $-367,60